In [1]:
from DRP.recommender.SampleGridParameters import GridValueGenerator

In [2]:
from DRP.recommender.ReactionGenerator import ReactionGenerator

In [3]:
a = GridValueGenerator().main()
react_gen = ReactionGenerator(*a)

In [4]:
react_gen

In [5]:
example_list = []

In [6]:
count = 0
for i in react_gen.generate():
    count+=1
    if count >= 100:
        break
    else:
        example_list.append(i)

TypeError: generate() takes exactly 2 arguments (1 given)

In [ ]:
from DRP.models import OrdRxnDescriptor

In [ ]:
for i in OrdRxnDescriptor.objects.all():
    print i.pk, i

In [ ]:
from DRP.models import NumRxnDescriptorValue
from DRP.models import OrdRxnDescriptorValue

In [ ]:
NumRxnDescriptorValue.objects.filter(reaction = example_list[0])

In [ ]:
initial_len = len(NumRxnDescriptorValue.objects.all())

In [ ]:
count = 0
for i in react_gen.generate():
    count+=1
    if count >= 2:
        break
    else:
        example_list.append(i)
        
assert(initial_len < len(NumRxnDescriptorValue.objects.all()))

In [ ]:
initial_len

In [ ]:
len(NumRxnDescriptorValue.objects.all())

In [ ]:
len(NumRxnDescriptorValue.objects.all())-initial_len

In [ ]:
example_list[-1]

In [ ]:
example_list[:7]

In [ ]:
ex_id_list = [rxn.id for rxn in example_list]

In [ ]:
from DRP.models import OrdRxnDescriptorValue, BoolRxnDescriptorValue
for i in BoolRxnDescriptorValue.objects.all():
    print i.pk, i.descriptor, i.descriptor.pk, i, i.reaction
    if i.pk > 90111:
        break

In [ ]:
BoolRxnDescriptorValue.objects.get(descriptor__pk=2, reaction__pk=27045)

In [ ]:
from DRP.models import OrdRxnDescriptorValue, BoolRxnDescriptorValue

class ReactionSieve(object):

    def __init__(self, model_container, reactions, desired_desc_dict):
      self.model_container = model_container
      self.desired_desc_dict = desired_desc_dict
    
    def filter(self, reactions):
        print 'TYPE', type(reactions)
        print '''ENTERING FILTER'''
        a = self.model_container.predict(reactions)#, verbose=True)
        print '''PREDICTIONS DONE'''
        plausible_reactions = []
        print '''DESIRED DESCRIPTORS:''', self.desired_desc_dict
        for desc, desc_vals in self.desired_desc_dict.items():
            for i in desc_vals:
                 for a in a[i]:
                     plausible_reactions.append(a[0])
#         print '''PLAUSIBLE REACTIONS:''', plausible_reactions
        plausible_reactions_ids = [pr.id for pr in plausible_reactions]
        reactions = reactions.filter(pk__range =(min(plausible_reactions_ids), max(plausible_reactions_ids)))
        print '''EXITING FILTER'''
        return reactions

        
        """
        # I think this was based on a misunderstanding
        for desc, desc_vals in self.desired_desc_dict.items():
#             print 'desc', desc, type(desc), desc_vals, type(desc_vals)
            plausible_ids = []
            #TODO MAKE WORK FOR NON-ORDINAL STUFF
            #reactions = [reaction for reaction in reactions if BoolRxnDescriptorValue.objects.get(descriptor__pk=2, reaction__range(min(ex_id_list), max(ex_id_list)))]
            for i in desc_vals:
                plausible_ids += [descriptor.reaction.id for descriptor in BoolRxnDescriptorValue.objects.filter(descriptor__pk=2) if descriptor.value==i]
            reactions = [reaction for reaction in reactions if reaction.id in plausible_ids]
            print 'plausible', plausible_ids
            print 'reactions', reactions
    
            #         return reactions
        """

In [ ]:
from DRP.models import ModelContainer

In [ ]:
from DRP.models import OrdRxnDescriptor
OrdRxnDescriptor.objects.get(pk=3)

In [ ]:
from DRP.recommender.SampleGridParameters import GridValueGenerator

In [ ]:
a = ReactionSieve(ModelContainer.objects.get(pk=1), Reaction.objects.filter(pk__range=(211890, 211990)), {BoolRxnDescriptor.objects.get(pk=2):[True]})

In [ ]:
a.filter(Reaction.objects.filter(pk__range=(211890, 211990)))

In [ ]:
print type(Reaction.objects.filter(pk__range=(211890, 211990)))

In [ ]:
reactions = [Reaction.objects.filter(pk__range = (min(ex_id_list),max(ex_id_list)))]

In [ ]:
import numpy as np
import pandas as pd

from DRP.recommender.AbstractRecommender import AbstractRecommender
from DRP.models import PerformedReaction
from DRP.models import CompoundQuantity


class Recommender(AbstractRecommender):
    def __init__(self, model_container, grid_params, desired_desc_dict):
        """
        All Descriptor Dicts use (Descriptor, Value List) pairs, where the
        Value List is a list of any values this Descriptor should have.
        """
        
        self.generator = ReactionGenerator(*grid_params)
        self.plausible_reactions = self.generator.generate()
        reaction_ids = []
        
        for i in range(10000):
            reaction_ids.append(self.plausible_reactions.next().pk)
            
        react_min = min(reaction_ids)
        react_max = max(reaction_ids)
        react_range = (react_min, react_max)
        self.sieve = ReactionSieve(model_container, Reaction.objects.filter(pk__range=react_range), desired_desc_dict)
        self.plausible_reactions =  Reaction.objects.filter(pk__range=react_range)

        self.plausible_reactions = self.sieve.filter(Reaction.objects.filter(pk__range=react_range))
        self.committed = []
        
        # TODO remove
        self.desc_dict = grid_params[0]
        self.compound_dict = grid_params[1]

    def array_of_proposed_rxns(self):

        self.existing_reactions = PerformedReaction.objects.all()
        reac_descs = {}
        df = pd.DataFrame(columns=self.compound_dict.keys() + self.desc_dict.keys())
        
        
        # TODO figrue out how to deal with compound ammounts
        for i in self.compound_dict:
            compounds = []
            for reaction in self.plausible_reactions:
                compound_options = CompoundQuantity.objects.filter(reaction=reaction)
                compounds += [compound.compound.name for compound in compound_options if Compound.objects.get(pk=compound.compound.pk) in self.compound_dict[i]]                
            df[i] = pd.Series(compounds)
        
#         for i in self.desc_dict:
#             desc_vals = []
#             for reaction in self.plausible_reactions:
#                 # TODO Expand beyond numeric descriptors
#                 desc_vals.append(NumRxnDescriptorValue.objects.get(reaction=reaction,descriptor=i).value)
#             vals_to_head = np.array(desc_vals)
#             df[i] = pd.Series(vals_to_head)
                        
        return df.as_matrix()
        
    def build_matrix(self):
        pass
        
        
    def mutual_info(self):
        pass

    def recommend(self):
        rxns = self.generator.generate()
        desired_rxns = self.sieve.filter(rxns)
        return desired_rxns
    
    
    

In [ ]:
a = Recommender(ModelContainer.objects.get(pk=1),  GridValueGenerator().main(), {BoolRxnDescriptor.objects.get(pk=2):[True]})

In [ ]:
matrix = a.array_of_proposed_rxns()

In [ ]:
for i in matrix[0:500:20]:
    print i

In [ ]:
def calc_MI(x, y, bins=5):
    c_xy = np.histogram2d(x, y, bins)[0]
    g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")
    mi = 0.5 * g / c_xy.sum()
    return mi

In [ ]:
def find_least_MI(recommender, outcome_descriptor):
    mutual_info_tuples = []
    base_matrix = recommender.create_matrix()
    for reaction in recommender.plausible_reactions:
        the_array = []
        for compounds in recommender.compound_dict:
            compound_options = CompoundQuantity.objects.filter(reaction=reaction)
            the_array += [compound.compound.name for compound in compound_options if Compound.objects.get(pk=compound.compound.pk) in recommender.compound_dict[compounds]]

            desc_vals = []
        for descriptor in recommender.desc_dict:
            desc_vals.append(NumRxnDescriptorValue.objects.get(reaction=reaction,descriptor=descriptor).value)
        
        OrdRxnDescriptor.objects.get(pk=3)
        
        the_array += desc_vals
        matrix2 = np.vstack([base_matrix,np.array(the_array)])
#         matrix2 = the_array
        mutual_info_tuples.append(calc_MI(base_matrix,matrix2))
        return mutual_info_tuples
            

In [ ]:
z = find_least_MI(a)

In [ ]:
print z

In [ ]:
from DRP.models import OrdRxnDescriptor

In [ ]:
for i in OrdRxnDescriptor.objects.all():
    print i.pk, i

In [ ]:
OrdRxnDescriptor.objects.get(pk=3)